Universidad Galileo  
Maestría en Data Science  
Statistical Learning I
Sección U  


   # **Tarea 1: Gradient descent en Tensorflow vectorizado**


<p style="text-align: right;">
Henry Giovanni<br/>
Barrientos García<br/>
21001538<br/>
Guatemala, 27 de mayo de 2022<br/>
</p>

In [1]:
import warnings
warnings.filterwarnings('ignore')
import datetime, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
%load_ext tensorboard
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


AttributeError: module 'tensorflow._api.v1.compat.v1.compat' has no attribute 'v1'

In [2]:
# Carga de datos
data = np.load('data/proyecto_training_data.npy')
data = pd.DataFrame(data)
data.columns = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"]
data.describe()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1201.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,70.049958
std,79442.502883,1.382997,386.587738,1.625393,30.202904,24.284752
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,59.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,69.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,80.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


In [3]:
# Division de datos para entrenamiento y pruebas
data = data.sample(frac = 1)
index = int(0.8 * len(data))
df_train = data[:index]
df_test = data[:-index]

# Modelo de regresión lineal con Tensorflow

A continuación crearemos una clase que será útil para la realización del modelo determinado, esta clase realiza las ejecuciones basadas en operaciones con Tensorflow.


In [4]:
class MRL:

    def __init__ (self):
        self.parameters = tf.get_variable("parametros", dtype = tf.float32, shape = [1,2], initializer = tf.zeros_initializer())
    
    # Ejecución de Gradient descent.
    # Devuelve el conjunto de valores útiles para una iteración de experimento
    def gradient_descent(self, x, y, lr):
        mse = 1/2 * tf.reduce_mean(tf.math.square(y - tf.matmul(self.parameters,x)))
        error_summary = tf.summary.scalar("mse_summary", mse)
        gradient = tf.gradients(mse, [self.parameters])
        new_w = tf.assign(self.parameters, self.parameters -lr * gradient[0])
        return new_w, mse, error_summary

A continuación implementaremos el entrenamiento del modelo y sus calculos.

**OverallQual** es la variable que utilicé en mi proyecto anterior, ésa fue con la que encontré una mayor conrrelación con la variable a predecir: **SalePrice**.

In [5]:
#Modelo de entrenamiento con sus hiperparametros
def training_model(x, y, lr, epochs, printl):
    # Experimento
    exp_string = './logs/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_epochs=" + str(epochs) + "_lr=" + str(lr)

    graph = tf.Graph()
    with graph.as_default():
        # tensores input
        tx = tf.placeholder(tf.float32, [2, len(df_train["OverallQual"])], "tx")
        ty = tf.placeholder(tf.float32, [len(df_train["SalePrice"])], "ty")
        
        # Nodo para cálculo de nuevos parámetros para GD
        mrl = MRL()
        current_parameters = mrl.gradient_descent(tx, ty, lr)
        
        # Writer para utilizacion de tensorboard
        file_writer = tf.summary.FileWriter(exp_string, graph)
        
        with tf.train.MonitoredSession() as session:
            # Paso de valores a los tensores input
            tensor_dict = {tx:x, ty:y}
            
            # Ejecución de epochs + 1 veces del algoritmo
            for i in range(epochs + 1):
                # Entrenamiento del modelo para cálculo de los nuevos parámetros
                model_training = session.run(current_parameters, feed_dict = tensor_dict)
                # Impresión de epochs
                if (i % printl == 0):
                    parameters = session.run(mrl.parameters, feed_dict = tensor_dict)
                    # Agregar datos a writer para poder visualizarlos en tensorboard
                    file_writer.add_summary(model_training[2], i)
                    print("Epoch: ", i, "Parámetros: ", parameters, "mse: ", model_training[1])
                    
            file_writer.close()

## Cálculo de experimentos

### Hipótesis

Según el algoritmo de Gradient Descent para alcanzar el error mínino (MSE) se depende del Learning rate, lo cual define la cantidad de epochs que son necesarios para alcanzar la convergencia.

Se estima que el lr debe estar comprendido entre alguno de los valores siguientes: 0.01, 0.02, 0.03, 0.04, 0.05, 0.06

In [6]:
x = df_train["OverallQual"]
x = np.array([x, np.ones_like(x)], dtype = "float64")
y = df_train["SalePrice"]

## Ejecución de experimentos
A continuación realizaremos las llamadas de ejecución al modelo de entrenamiento definido anteriormente para los calculos que serán utiles para trabajar con Tensorboard.

In [7]:
iterations = 6
lrs = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]
epochss = [24000, 22000, 10000, 10000, 10000, 10000]
printls = [1500, 1500, 1000, 1000, 1000, 1000]
for k in range(iterations):
    print("\n*** EXPERIMENT {}: lr={}, epochs={}, println={}***".format(k, lrs[k], epochss[k], printls[k]))
    print("--------------------------------------------------------")
    training_model(x, y, lrs[k], epochss[k], printls[k])


*** EXPERIMENT 0: lr=0.01, epochs=24000, println=1500***
--------------------------------------------------------
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Parámetros:  [[11933.457   1812.6871]] mse:  19629758000.0
Epoch:  1500 Parámetros:  [[ 38161.332 -49265.496]] mse:  1256514300.0
Epoch:  3000 Parámetros:  [[ 42357.625 -76134.164]] mse:  1205205000.0
Epoch:  4500 Parámetros:  [[ 44450.97  -89537.734]] mse:  1192436500.0
Epoch:  6000 Parámetros:  [[ 45495.234 -96224.13 ]] mse:  1189259100.0
Epoch:  7500 Parámetros:  [[ 46016.17 -99559.67]] mse:  1188468100.0
Epoch:  9000 Parámetros:  [[  46276.047 -101223.6  ]] mse:  1188271400.0
Epoch:  10500 Parámetros:  [[  46405.676 -102053.64 ]] mse:  1188222200.0
Epoch:  12000 Parámetros:  [[  46470.355 -102467.77 ]] mse:  1188210200.0
Epoch:  13500 Parámetros:  [

In [8]:
%tensorboard --logdir ./logs

Reusing TensorBoard on port 6006 (pid 98544), started 3 days, 10:01:33 ago. (Use '!kill 98544' to kill it.)

## Conclusiones

1. Se puede observar que el error mínimo se alcanza cuanto más grande es el _Learning rate_, esto basandonos en las observaciones del comportamiento del MSE respecto del _Learning rate_.

2. Luego de la ejecución de los experimentos, se observa que el Learning rate tiene una relación directamente proporcional con los pasos de _Gradient descent_, lo que implica tener que utilizar menos _epochs_.

3. Al avanzar con los experimentos y hacer más grande el valor del _Learning rate_, podemos observar cómo el algortimo _Gradient descent_ llega a un resultado indefinido, esto seguramente porque no logra alcanzar la convergencia.

4. Como conclusión para nuestra experimentación vemos cómo para alcanzar la convergencia de forma rápida y con menos epochs, es requerido definir los hiperparámetros como: **lr = 0.04**, con lo cual obtenemos el error minímo aproximadamente en el **epoch = 4000**.